# Optymalizacja portfela papierów wartościowych

Celem projektu jest zbadanie zastosowań algorytmów ewolucyjnych do optymalizacji portfela papierów wartościowych, jako problemu optymalizacji wielokryterialnej.

## Opis zagadnienia
Optymalizacja portfela polega na minimalizacji (lub maksymalizacji) różnych statystyk, obliczanych na podstawie historycznych zwrotów takich jak średnia, czy też wariancja.
W praktyce równie istotne jest sprawdzenie, jak sprawdza portfel dobrany poprzez taką optymalizację, jednak nie będziemy się zajmować testowaniem działania otrzymanych portfeli w przyszłości.

Optymalizacja portfela polega na znalezieniu wag portfela o najmniejszym ryzyku dla ustalonego wcześniej poziomu zwrotu, lub wag portfela o największym zwrocie dla ustalonego, maksymalnego poziomu ryzyka.
Ze względu na to, że istnieją różne miary ryzyka, będziemy szukali odpowiedniego frontu Pareto, który dla określonego poziomu zwrotu będzie minimalizował wszystkie zaimplementowane przez nas miary ryzyka.

Klasycznym podejściem do tego problemu jest teoria portfelowa Markowitza, która pozwala na znalezienie optymalnego portfela, jeśli za miarę ryzyka przyjmiemy wariancję zwrotów.
Z różnych względów wykorzystywanie wariancji, jako miary ryzyka jest krytykowane przez praktyków zajmujących się finansami, natomiast dla innych miar ryzyka nie występują tak zwarte procedury, pozwalające na bezpośrednie obliczenie optymalnych wag, dlatego też próba optymalizacji z wykorzystaniem algorytmów ewolucyjnych jest uzasadniona.

Będziemy rozpatrywali rynek, który składa się z $d$ instrumentów finansowych.
Powiemy, że $R=(R_1, \dots, R_d)^T$ jest wektorem losowym stóp zwrotu odpowiednich instrumentów, a $\mu=\mathbb{E}(R)$ jest jego wartością oczekiwaną.
Interesuje nas znalezienie wektora wag $w=(w_1, \dots, w_d)^T$ takiego, że $\sum_{i=1}^d w_i = 1$, dla którego wartość oczekiwana $\mathbb{E}(w^TR) = w^T \mu$ jest jak największa.

Na podstawie danych rynkowych estymujemy $\hat{\mu}_i = \bar{r^{(i)}} = \sum_{t=1}^T r^{(i)}_t$, jako średnią dzienną stopę zwrotu.

Dlatego też w praktyce będziemy szukać takich wag, które maksymalizują $w^T\hat{\mu}$.

Podstawową miarą ryzyka jest wariancja $\text{Var}(w^TR) = w^T\text{Cov}(R) w = w^T \Sigma w$, gdzie $\Sigma$ jest macierzą kowariancji wektora $R$.
W naszym przypadku będziemy wykorzystywać estymator macierzy kowariancji $\hat{\Sigma} = \frac{1}{T}\sum_{t=1}^T (r_t - \mu) (r_t - \mu)^T.$
Zależy nam zatem na minimalizacji wyrażenia $w^T \hat{\Sigma} w$.

Kolejną miarą ryzyka, którą będziemy wykorzystywać jest semiwariancja.
Dla portfela o wagach $w$ jest ona określona, jako $s=\mathbb{E}\left((w^TR - \mathbb{E}(w^TR))^2 1_{\{w^TR \leq \mathbb{E}[w^TR]\}}\right)$,
Podobnie, jak w innych przypadkach, będziemy estymować tę wartość na podstawie historycznych zwrotów, jako $\hat{s}=\frac{1}{|\{t: w^Tr_t < w^T\mu\}|} \sum_{t: w^Tr_t < w^T\mu} (w^T\mu - w^Tr_t)^2$.

Ostatnią miarą ryzyka, którą wykorzystamy jest VaR (Value at Risk) zdefiniowana, jako $\text{VaR}_\alpha(w^TR) = \inf\{ x:P(w^TR + x < 0) \leq \alpha \}.$
Tutaj również będziemy korzystać z estymatora tego parametru, uzyskanego na podstawie danych historycznych,
danego wzorem $\hat{\text{VaR}_\alpha} = \text{kwantyl rzędu }(1 - \alpha) \text{ ze zbioru wartości } \{ w^Tr_t: t=1,\dots,T\}.$

## Funkcja celu
Poprzez optymalizację portfela w kontekście tego projektu będziemy rozumieć minimalizację wariancji, semiwariancji, $\hat{\text{VaR}_{95\%}}$ i $\hat{\text{VaR}_{99\%}}$ i jednoczesną maksymalizację oczekiwanego zwrotu. Dla uproszczenia obliczeń, będziemy minimalizować wszystkie miary ryzyka oraz negatywny zwrot.

Dla wszystkich zbiorów akcji zostały wykorzystane dzienne zwroty dla okresów czasu o różnej długości, zależnej od problemu. Przez dzienne rozumiemy tutaj stopę zwrotu między kolejnymi dniami biznesowymi.

Rozpatrujemy funkcję celu, która wektorowi wag $w$ takiemu, że $\sum_{i=1}^d w_i = 1$ przypisuje wektor składający się z negatywnego zwrotu, wariancji, semiwariancji, $\hat{\text{VaR}_{95\%}}$ i $\hat{\text{VaR}_{99\%}}$.

Funkcją celu jest
$$f(w) = (-w^T\mu, w^T \hat{\Sigma} w, \hat{s}, \hat{\text{VaR}_{95\%}}, \hat{\text{VaR}_{99\%}}),$$
natomiast przestrzenią poszukiwań jest hiperpłaszczyzna w $\mathbb{R}^d$ wyznaczona przez warunek $\sum_{i=1}^d w_i = 1$.

Jest to zatem klasyczny problem optymalizacji wielokryterialnej.

## Algorytmy ewolucyjne

Do optymalizacji wielokryterialnej wykorzystałem zaimplementowany w ramach projektu algorytm NSGAII.
W trakcie eksperymentów wykorzystane zostały operatory krzyżowania: SBX (Simulated Binary Crossover), Uniform Crossover oraz PMX (Partially Mapped Crossover). Operatory mutacji, które były testowane w optymalizacji portfela to Polynomial Mutation, ale też Reverse Sequence Mutation.
Po każdym krzyżowaniu oraz mutacji wszystkie wektory wag były normalizowane, aby ich suma wynosiła 1.

W trakcie eksperymentów wystąpił również problem zbyt szerokich frontów Pareto, które nie są interesujące z praktycznej perspektywy - algorytm doprowadzał do rozwiązań o bardzo dużej stopie zwrotu, lecz przy nieakceptowalnym poziomie ryzyka. Dlatego też podczas eksperymentów została sztucznie narzucana górna granica na możliwy zwrot z portfela.

## Wyniki

Dla każdego ze zbiorów szukamy wag, wykorzystując teorię portfela Markowitza, a wartości miar ryzyka dla tak otrzymanych wag wykorzystujemy jako podstawę do oceny wyników działania algorytmów ewolucyjnych.
Chcielibyśmy, aby dla określonego poziomu zwrotu optymalizacja portfela z wykorzystaniem algorytmu NSGAII dawała rezultaty niewiele gorsze pod względem wariancji i jednocześnie lepsze pod względem pozostałych miar ryzyka.

### WIG20
W pierwszym eksperymencie zbadałem działanie algorytmu dla zbioru akcji z indeksu WIG20 (wg stanu na 30.04.2009). Pod uwagę były brane notowania między 2008 a 2022 rokiem.
Wykorzystując UniformCX oraz Polynomial Mutation udało się otrzymać front Pareto, który pokrywa się z frontem Pareto otrzymanym na podstawie teorii portfela Markowitza.
Co więcej, uzyskane wskaźniki VaR są nieznacznie mniejsze niż te, otrzymane z wykorzystaniem metody Markowitza.

![](output/WIG20.png)